In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import pickle
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def convert_to_polar(df) -> pd.DataFrame:
    r = np.sqrt(df['direction_x']**2 + df['direction_y']**2 + df['direction_z']**2)
    df['zenith'] = np.arccos(df['direction_z']/r)
    df['azimuth'] = np.arctan2(df['direction_y'],df['direction_x'])
    df['azimuth'][df['azimuth']<0] = df['azimuth'][df['azimuth']<0] + 2*np.pi 

    return df[['azimuth', 'zenith', 'direction_kappa']]

In [ ]:
def convert_to_xyz(df) -> pd.DataFrame:
    df['x'] = np.sin(df['zenith'])*np.cos(df['azimuth'])
    df['y'] = np.sin(df['zenith'])*np.sin(df['azimuth'])
    df['z'] = np.cos(df['zenith'])
    return df[['x', 'y', 'z']]

In [ ]:
with sqlite3.connect('data/F4/focus_batch_4.db') as con:
        query = 'select * from meta_table'
        meta_df = pd.read_sql(query,con)

meta_df = meta_df[['event_id', 'azimuth', 'zenith']].set_index('event_id')

In [ ]:
target_df = convert_to_xyz(meta_df)

In [ ]:
target_df_copy = target_df.copy()
target_df_copy.rename(columns={'x':'target_x', 'y':'target_y', 'z':'target_z'}, inplace=True)

In [ ]:
p0 = pd.read_pickle('inference/pred_M0_F4.pkl')
p0.index = p0.index.astype(int)

p1 = pd.read_pickle('inference/pred_M1_F4.pkl')
p1.index = p1.index.astype(int)

p2 = pd.read_pickle('inference/pred_M2_F4.pkl')
p2.index = p2.index.astype(int)

p3 = pd.read_pickle('inference/pred_M3_F4.pkl')
p3.index = p3.index.astype(int)

p0.rename(columns={'direction_x':'x0', 'direction_y':'y0', 'direction_z':'z0', 'direction_kappa':'k0'}, inplace=True)
p1.rename(columns={'direction_x':'x1', 'direction_y':'y1', 'direction_z':'z1', 'direction_kappa':'k1'}, inplace=True)
p2.rename(columns={'direction_x':'x2', 'direction_y':'y2', 'direction_z':'z2', 'direction_kappa':'k2'}, inplace=True)
p3.rename(columns={'direction_x':'x3', 'direction_y':'y3', 'direction_z':'z3', 'direction_kappa':'k3'}, inplace=True)

p0.shape, p1.shape, p2.shape, p3.shape

In [ ]:
data = pd.concat([p0, p1, p2, p3, target_df_copy], axis=1)

In [ ]:
train = data.sample(frac=0.90, random_state=0)
validation = data.drop(train.index)
train.reset_index(inplace=True, drop=True)
validation.reset_index(inplace=True, drop=True)

train.shape, validation.shape

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler


In [ ]:
# Prepare dataset
class CustomDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

# Define neural network
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, output_size)
        )

    def forward(self, x):
        return self.layers(x)


In [ ]:
n_features = train.shape[1] - 3
batch_size = 2048
# Prepare data
train_data = train.iloc[:, :n_features].values
train_targets = train.iloc[:, n_features:].values
validation_data = validation.iloc[:, :n_features].values
validation_targets = validation.iloc[:, n_features:].values

# Convert to PyTorch tensors
train_data = torch.tensor(train_data, dtype=torch.float32)
train_targets = torch.tensor(train_targets, dtype=torch.float32)
validation_data = torch.tensor(validation_data, dtype=torch.float32)
validation_targets = torch.tensor(validation_targets, dtype=torch.float32)

# Create data loaders
train_dataset = CustomDataset(train_data, train_targets)
validation_dataset = CustomDataset(validation_data, validation_targets)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NeuralNetwork(n_features, 3).to(device)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 20000
min_validation_loss = float('inf')
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for batch, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)

    # Validation
    model.eval()
    with torch.no_grad():
        validation_loss = 0
        for batch, (inputs, targets) in enumerate(validation_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            validation_loss += loss.item()

        validation_loss /= len(validation_loader)

    if validation_loss < min_validation_loss:
        min_validation_loss = validation_loss
        torch.save(model, "ensemble/ensemble.pth")
        print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {validation_loss:.4f}, Model Saved")

    else:    
        print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {validation_loss:.4f}")